# 개발 지침 약어

### DRY/OAOO
- DRY: Do not Repeate Yourself
- OAOO: Once and Only Once  

코드에 있는 지식은 단 한번, 단 한 곳에 정의되어야 함  

**코드 중복으로 인해 생기는 문제점**
1. 오류가 발생하기 쉬움
1. 비용이 비쌈: 변경하는 데 더 많은 시간이 소요됨  
1. 신뢰성이 떨어짐  
---
코드 중복으로 인한 나쁜 코드의 예

In [1]:
def process_students_list(students):
    students_ranking = sorted(students, key=lambda s: s.passed * 11 - s.failed * 5 - s.years * 2)
    
    # 학생별 순위 출력
    for student in students_ranking:
        print(
            "이름: {0}, 점수: {1}".format(
                student.name, 
                (student.passed * 11 - student.failed * 5 - student.years * 2)
            ))

sorted 함수의 key로 사용되는 lambda가 특별한 도메인 지식을 나타내지만 아무런 정의가 없음  
도메인 문제에 대한 지식이 사용된 경우 의미를 부여해야하므로 아래와 같이 코드를 바꾸는 것이 도움이 됨

In [2]:
def score_for_student(student):
    return student.passed * 11 - student.failed * 5 - student.years * 2

def process_students_list(students):
    student_ranking = sorted(students, key=score_for_student)
    
    #학생별 순위 출력
    for student in student_ranking:
        print(
            "이름: {0}, 점수: {1}".format(
                student.name, score_for_student(student)
            ))

### YAGNI
= You Ain't Gonna Need It  
= 굳이 필요 없는 추가 개발 하지 않기
- 과잉 엔지니어링을 하지 않기 위해 솔루션 작성 시 계속 염두에 두어야 하는 원칙
- 유지보수가 간으한 소프트웨어를 만드는 것은 미래의 요구사항을 예측하는 것이 아님  
- 현재의 요구사항을 잘 해결하기 위한 소프트웨어를 작성하고 가능한 나중에 수정하기 쉽도록 작성하기

### KIS
= Keep It Simple  
=  소프트웨어 컴포넌트 설계 시 과잉 엔지니어링을 피하기
- 문제를 올바르게 해결하는 최소한의 기능을 구현  

아래 클래스는 제공된 키워드 파라미터 세트에서 네임 스페이스를 작성하지만 복잡한 코드 인터페이스를 가짐

In [5]:
class ComplicatedNamespace:
    """프로퍼티를 가진 객체를 초기화하는 복잡한 예제"""
    
    ACCEPTED_VALUES = ("id_", "user", "location")
    
    @classmethod
    def init_with_data(cls, **data):
        instance = cls()
        for key, value in data.items():
            if key in cls.ACCEPTED_VALUES:
                setattr(instance, key, value)
        return instance

객체를 초기화하기 위해 추가 클래스 메서드를 만드는 것은 불필요  
반복을 통해 setattr을 호출하는 것 역시 이상

In [7]:
cn = ComplicatedNamespace.init_with_data(id_=42, user="root", location="127.0.0.1", extra="excluded")
print(cn.id_, cn.user, cn.location)
print(hasattr(cn, "extra"))

42 root 127.0.0.1
False


초기화를 위해 init&#95;with&#95;data 라는 일반적이지 않은 메서드의 이름을 알아야 하는 것도 불편한 부부  
파이썬에서 다른 객체를 초기화 할 때는 &#95;&#95;init&#95;&#95;메서드를 사용하는 것이 간편  

---
위의 예제는 아래처럼 바꾸는 것이 적절

In [8]:
class Namespace:
    """Create an object from keyword arguments"""
    
    ACCEPTED_VALUES = ("id_", "user", "location")
    
    def __init__(self, **data):
        accepted_data = {
            k: v for k, v in data.items() if k in self.ACCEPTED_VALUES
        }
        self.__dict__.update(accepted_data)

### EAFP/LBYL
- EAFP: Easier to Ask Forgiveness than Permission  
- LBYL: Look Before You Leap  

**EAFP**는 일단 코드를 실행하고 실제 동작하지 않을 경우에 대응한다는 뜻  
일반적으로는 코드를 실행하고 발생한 예외를 catch하고 except 블록에서 바로잡는 코드를 실행  

**LBYL**은 도약하기 전에 무엇을 사용하려고 하는 지를 확인하라는 뜻  
e.g) 파일을 사용하기 전에 먼저 파일을 사용할 수 있는 지를 확인
```python
if os.path.exists(filename):
    with open(filename) as f:
        ...
```

위의 방식은 EAFP방식으로 만들어진 파이썬스러운 방식은 아님  
따라서 위의 코드를 아래와 같이 바꾸는 것이 적절  
```python
try:
    with open(filename) as f:
        ...
except FileNotFoundError as e:
    logger.error(e)
```